In [1]:
# import pandas as pd
# answer_df = pd.read_csv("../../data2/answer_df_llama.csv")
# answer_df.iloc[0:500].to_csv("../../data2/answer_df_llama1.csv", index=False)
# answer_df.iloc[500:1000].to_csv("../../data2/answer_df_llama2.csv", index=False)
# answer_df.iloc[1000:].to_csv("../../data2/answer_df_llama3.csv", index=False)


In [2]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print(torch.cuda.is_available())

True


In [3]:

model_name = "meta-llama/Llama-3.1-8B-Instruct"  # Replace with exact path for 8B model
tokenizer = AutoTokenizer.from_pretrained(model_name)

print(torch.cuda.is_available())

# Check if CUDA is available
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU

# Load the model with the appropriate device settings
model = AutoModelForCausalLM.from_pretrained(
    model_name,                   
    return_dict=True,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,     # Use float16 for efficient memory usage on GPU
    device_map="auto" if device == 0 else None,  # Automatically distribute across devices or use CPU
    trust_remote_code=True
)

# Initialize the text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto" if device == 0 else None,  # Explicitly set device to GPU if available, else CPU
)
# For model_0
for name, param in model.named_parameters():
    print(f"{name} is on {param.device}")



True


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

model.embed_tokens.weight is on cuda:0
model.layers.0.self_attn.q_proj.weight is on cuda:0
model.layers.0.self_attn.k_proj.weight is on cuda:0
model.layers.0.self_attn.v_proj.weight is on cuda:0
model.layers.0.self_attn.o_proj.weight is on cuda:0
model.layers.0.mlp.gate_proj.weight is on cuda:0
model.layers.0.mlp.up_proj.weight is on cuda:0
model.layers.0.mlp.down_proj.weight is on cuda:0
model.layers.0.input_layernorm.weight is on cuda:0
model.layers.0.post_attention_layernorm.weight is on cuda:0
model.layers.1.self_attn.q_proj.weight is on cuda:0
model.layers.1.self_attn.k_proj.weight is on cuda:0
model.layers.1.self_attn.v_proj.weight is on cuda:0
model.layers.1.self_attn.o_proj.weight is on cuda:0
model.layers.1.mlp.gate_proj.weight is on cuda:0
model.layers.1.mlp.up_proj.weight is on cuda:0
model.layers.1.mlp.down_proj.weight is on cuda:0
model.layers.1.input_layernorm.weight is on cuda:0
model.layers.1.post_attention_layernorm.weight is on cuda:0
model.layers.2.self_attn.q_proj.w

In [4]:
piece_id = 1
answer_df_path = "../../data2/answer_df_llama"+str(piece_id)+".csv"
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data2/fea_df.csv")
if 'answer_string' not in answer_df.columns:
    answer_df['answer_string'] = np.nan
print(answer_df.shape)

query_df

(500, 3)


,fea,description
0,weightbasedhealth,Judgments about health based on weight or appe...
1,weightstigma,"Negative attitudes, discrimination, or prejudi..."
2,weightbasedhealthcare,Healthcare decisions influenced by weight. Not...
3,encourageweightloss,Encouragement of weight loss
4,dietpromotion,Advocacy for diets or restrictive eating
5,thinness,Reassurance of thinness. Maintaining thinness
6,notlowweightenough,"Not sick enough, or not thin enought, or not l..."
7,weightblame,Weight blamed for health issues or concerns
8,weightvalue,Weight tied to personality characteristics


In [5]:
from time import time


system_message = """
You are an AI assistant designed to answer questions.
Use the exact answer format asked.
"""


def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

def format_prompt(text):
    question_content = "The provided parapraph is eating disorders patients' experiences with physicians. Weight stigma is indicated by weight-or-appearance-related treatment or judgement in this context. Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: [yes/no], related phrases if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = question_content + answer_content + paragragh_content
    #print(user_message)
    
    return user_message



def query_model_batch(
        system_message,
        user_messages,
        temperature=0,
        max_length=1024
    ):
    start_time = time()
    # Add "Question: ... Answer:" to each user message for clarity
    batched_messages = [
        "Question: " + message + " Answer:" for message in user_messages
    ]
    
    # Construct prompts for each message in batch
    all_prompts = [
        pipe.tokenizer.apply_chat_template(
            [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            tokenize=False,
            add_generation_prompt=True
        ) for user_message in batched_messages
    ]
    
    # Define the end-of-sequence terminators
    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    
    # Run the batch inference
    sequences = pipe(
        all_prompts,
        do_sample=True,
        top_p=0.5,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=terminators[0]
    )
    
    # Extract generated text for each sequence
    answers = []
    for i, sequence in enumerate(sequences):
        answer = sequence[0]['generated_text']
        total_time = f"Total time: {round(time() - start_time, 2)} sec."
        # Format the response with timing information
        answers.append(batched_messages[i] + " " + answer + " " + total_time)

    return answers




In [6]:
for k in range(1000):
    batch_size = 10

    # Filter for rows where 'answer_string' is NaN
    unanswered_df = answer_df[answer_df['answer_string'].isna()]

    # Get the indices of these NaN entries in the original DataFrame
    indices_to_update = unanswered_df.index[:batch_size]

    # Prepare prompt content for the first 10 entries with NaN answer_string
    prompts = [format_prompt(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

    # Save the indices list if needed for later use
    indices_to_update_list = list(indices_to_update)


    # Batch process all prompts at once
    responses = query_model_batch(
        system_message=system_message,
        user_messages=prompts,
        temperature=0.1,
        max_length=512
    )

    if len(indices_to_update_list)>0:
        # Display and process responses in a loop
        for i, response in enumerate(responses):
            #display(Markdown(colorize_text(f"{response}")))

            # Extract answer if available
            if "Answer:" in response:
                answer = response.split("Answer:")[1]
                # Use the original index from indices_to_update_list
                answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer
            else:
                # Use the original index from indices_to_update_list
                answer_df.loc[indices_to_update_list[i], 'answer_string'] = "Answer not found"


        print(answer_df.loc[indices_to_update_list, :])
        answer_df.to_csv(answer_df_path, index=False)
    else:
         break


   sm_id                                         text_w_eos  \
0   1690  i saw a cardiologist twice and was told i did ...   
1    724       sharing my weight with me when asked not to.   
2   2800  'i do not think it is an eating disorder.'   a...   
3   1771  when i was severely underweight my pcp suggest...   
4   1975                primary care providers not educated   
5    126  the first time i sought a therapist they state...   
6   1511  unhelpful more than damaging more likely in th...   
7   4091  most of the 40 years i had a diagnosis of bn p...   
8    371  cut out food groups to help with physical disa...   
9   4313  primary care provider told me to 'go eat a che...   

                                       answer_string  
0   (1) weightbasedhealth: no, \n(2) weightstigma...  
1   (1) weightbasedhealth: no, \n(2) weightstigma...  
2   (1) weightbasedhealth: no, \n(2) weightstigma...  
3   (1) weightbasedhealth: yes, related phrases i...  
4   (1) weightbasedhealth: no, 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


    sm_id                                         text_w_eos  \
90   4760  insist on weighing you.  if you walk in with a...   
91     86  you can walk in with a broken arm and they tel...   
92    175  they told me my weight when i specifically say...   
93   3201                                       do you eat?!   
94   2523  positive comments when being weighed at primar...   
95    439  a provider caused harm by not bringing attenti...   
96    225  general practitioner kept telling me to gain w...   
97   3277  she told me that i was too sick and she did no...   
98    114    cardiologist told me to just eat a cheeseburger   
99   1093                                    too much to say   

                                        answer_string  
90   (1) weightbasedhealth: yes, related phrases i...  
91   (1) weightbasedhealth: no, \n(2) weightstigma...  
92   (1) weightbasedhealth: yes, related phrases i...  
93   (1) weightbasedhealth: no, \n(2) weightstigma...  
94   (1) weight

IndexError: list index out of range

In [7]:
import pandas as pd

# File paths
file_paths = [
    "../../data2/answer_df_llama1.csv",
    "../../data2/answer_df_llama2.csv",
    "../../data2/answer_df_llama3.csv"
]

# Read and concatenate dataframes
answer_df_list = [pd.read_csv(file_path) for file_path in file_paths]
answer_df = pd.concat(answer_df_list, ignore_index=True)

# Save the concatenated dataframe
output_path = "../../data2/answer_df_llama.csv"
answer_df.to_csv(output_path, index=False)
